# REDO AGE, MASS PLOTS

# Read in all the files, and make some plots

*Andrew Bowen provided some of the script to read in the file. [See his GitHub repo](https://github.com/andrewbowen19/CEB_Project)*

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde
import sys

from astropy.coordinates import SkyCoord
from astropy import units 

import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 500)

%matplotlib inline

In [2]:
#MWSC - Milky Way Star Clusters Catalog
#https://heasarc.gsfc.nasa.gov/W3Browse/all/mwsc.html

mwsc_df = pd.read_csv("MWSC.txt", header=3, delimiter='|')
mwsc_df.columns = mwsc_df.columns.str.strip()

#fix the IDs
mwsc_ID = mwsc_df['Name']
mwsc_ID = mwsc_ID.str.strip().str.replace( ' ','_' )
mwsc_df['Name'] = mwsc_ID

#take only the open clusters
mwsc_df = mwsc_df.loc[(~mwsc_df['class'].str.strip().str.contains('GLOBULAR')) &
                      (~mwsc_df['class'].str.strip().str.contains('NEBULA')) &
                      (~mwsc_df['class'].str.strip().str.contains('UNIDENTIFIED'))]

#remove the "Unnamed" columns
mwsc_df = mwsc_df.loc[:, ~mwsc_df.columns.str.contains('^Unnamed')]

print(set(mwsc_df['class'].values))

mwsc_df#.loc[mwsc_df['Name'] == 'PTB_9']

{'                          OB ASSOCIATION/HII REGION', '                                  OPEN STAR CLUSTER'}


,Name,broad_type,cluster_status,ra,dec,lii,bii,core_radius,central_radius,cluster_radius,pm_ra,pm_dec,pm_tot_error,rad_vel,rad_vel_error,num_rad_vel_stars,num_core_stars,num_central_stars,num_cluster_stars,distance,e_bv,distance_modulus,e_jk,e_jh,delta_h,log_age,log_age_error,num_log_age_stars,king_core_radius,king_core_radius_error,king_tidal_radius,king_tidal_radius_error,king_norm_factor,king_norm_factor_error,reference_code,cluster_type,metallicity,metallicity_error,num_metallicity_stars,comments,class
0,MWSC_4688,,,23 51 54,-86 43.2,303.907,-30.295,0.020,0.100,0.185,3.20,-5.00,1.13,,,,2,22,57,1336,0.219,10.700,0.105,0.070,0.000,9.390,,,1.05,0.39,7.01,2.51,2.51,0.67,AIPk,,,,0,...,OPEN STAR CL...
1,MWSC_5684,,,12 53 43,-86 38.9,302.968,-23.776,0.020,0.080,0.155,-13.04,0.17,1.05,,,,3,19,52,1432,0.375,10.900,0.180,0.120,0.020,9.180,0.023,3,0.61,0.42,7.54,5.79,1.88,1.09,ARIs,,,,0,...,OPEN STAR CL...
2,MWSC_5692,,,17 47 20,-86 36.6,306.562,-26.146,0.025,0.095,0.135,-6.22,-9.02,1.38,,,,4,18,28,1555,0.437,11.100,0.210,0.140,0.020,8.930,,,0.76,0.56,5.18,3.82,1.51,0.77,ARIs,,,,0,...,OPEN STAR CL...
3,MWSC_4005,,,00 11 28,-85 28.8,303.852,-31.577,0.012,0.100,0.165,9.31,-1.47,0.95,,,,3,26,42,1159,0.250,10.400,0.120,0.080,-0.020,9.375,,,0.36,0.13,4.69,1.85,15.94,6.19,AIPk,,,,0,...,OPEN STAR CL...
4,MWSC_4176,,,14 27 18,-85 25.2,304.950,-22.929,0.025,0.150,0.280,-9.41,0.12,0.69,,,,4,53,155,1093,0.333,10.300,0.160,0.107,-0.030,9.315,,,0.97,0.24,6.84,1.58,6.46,1.31,AIPk,,,,0,...,OPEN STAR CL...
5,ESO_008-06,r,c,14 56 55,-83 26.7,306.593,-21.485,0.025,0.130,0.185,-5.26,-4.21,0.80,,,,5,46,78,1380,0.312,10.800,0.150,0.100,0.030,9.300,,,0.66,0.25,5.60,2.18,5.93,1.91,DIAS,,,,0,"Sparse; center is shifted to 14.9485h,-83.445d...",OPEN STAR CL...
6,MWSC_4219,,,15 42 14,-83 11.7,307.905,-22.041,0.015,0.115,0.200,-9.26,-3.14,0.74,,,,2,36,95,1606,0.375,11.150,0.180,0.120,0.020,9.100,,,2.81,0.76,9.78,2.13,1.71,0.41,AIPk,,,,0,...,OPEN STAR CL...
7,MWSC_5575,,,01 59 42,-83 03.0,300.484,-33.751,0.015,0.090,0.150,6.33,-2.50,1.96,,,,1,10,20,2191,0.302,11.800,0.145,0.097,0.015,9.200,,,1.95,1.00,12.13,6.13,0.79,0.18,ARIs,,,,0,Poor RDP. ...,OPEN STAR CL...
8,MWSC_4682,,,23 43 23,-82 57.6,305.407,-33.838,0.020,0.115,0.190,5.87,0.28,0.94,,,,2,22,58,1065,0.354,10.250,0.170,0.113,0.000,9.280,0.061,4,0.43,0.24,8.27,5.39,5.33,3.02,AIPk,,,,0,...,OPEN STAR CL...
10,MWSC_5685,,,13 05 24,-82 02.6,303.443,-19.185,0.020,0.090,0.160,-3.12,-2.73,1.14,,,,3,25,64,1581,0.406,11.125,0.195,0.130,0.000,9.150,,,0.65,0.21,7.54,2.68,9.02,2.40,ARIs,,,,0,Poor RDP. ...,OPEN STAR CL...


### Downloaded from WEBDA [here](https://webda.physics.muni.cz/cluster_selall.html)

With RA from 0 to 24 and 0 to 1e6 stars. I copied the table to WEBDA.html, and removed the $<$br$>$ entries, then converted to csv with 

https://codepen.io/malahovks/pen/gLxLWX

or

https://jsfiddle.net/gengns/j1jm2tjx/

Finally, I separated the RA and DEC column header into 2 entries.

*I also have a data file from David James, that has more clusters, but I'm not sure the providence of that, so I won't use it.*

In [3]:
# # WEBDA data file (2013)
# webda_df = pd.read_fwf("WEBDA-OC-table-June2013_DavidJames.txt", 
#                        widths = [18,14,15,11,9,8,8,8,9,6,9,9,9,7,7,9], header = 0)

webda_df = pd.read_csv('WEBDA.csv')
#fix the IDs
webda_ID = webda_df['Name']
webda_ID = webda_ID.str.replace( 'NGC 0','NGC ' ).str.replace( ' ','_' )

webda_df['Name'] = webda_ID

webda_df

,Name,RA_2000,Dec_2000,l,b,Dist,Mod,EB-V,Age,ST,Z,Diam,Fe/H,MRV,pm RA,pm Dec,Measures,Stars
0,Berkeley_58,00 00 12,+60 58 00,116.753,-1.289,3715.0,14.55,0.550,8.400,NaN,-83.6,5.0,NaN,NaN,NaN,NaN,525,519
1,Stock_18,00 01 37,+64 37 30,117.624,2.268,2800.0,14.41,0.700,6.780,B0,110.8,6.0,NaN,NaN,NaN,NaN,2261,2261
2,Berkeley_59,00 02 13,+67 25 11,118.220,5.000,1000.0,13.78,1.220,6.800,NaN,87.2,20.4,NaN,-6.50,-4.40,0.73,27,21
3,Blanco_1,00 04 07,-29 50 00,15.572,-79.261,269.0,7.18,0.010,7.796,B5,-264.3,70.0,0.23,NaN,20.17,3.00,109,105
4,ASCC_1,00 09 35,+62 40 48,118.150,0.190,4000.0,13.51,0.160,8.250,NaN,13.3,24.0,NaN,-76.15,-2.07,0.46,32,32
5,Berkeley_1,00 09 36,+60 28 30,117.796,-1.979,2420.0,14.35,0.780,8.600,NaN,-83.6,5.0,NaN,NaN,NaN,NaN,2800,2800
6,King_13,00 10 06,+61 10 00,117.968,-1.306,3100.0,15.00,0.820,8.500,NaN,-70.7,5.0,NaN,NaN,NaN,NaN,4253,3955
7,Alessi_20,00 10 33,+58 45 35,117.640,-3.690,450.0,8.95,0.220,8.220,NaN,-29.0,36.0,NaN,NaN,7.48,-2.61,42,42
8,ASCC_2,00 19 51,+55 42 35,118.460,-6.890,1200.0,10.71,0.100,8.830,NaN,-144.0,36.0,NaN,NaN,-0.91,-3.94,57,57
9,Mayer_1,00 21 54,+61 44 24,119.440,-0.930,1429.0,12.02,0.400,7.740,NaN,-23.2,24.0,NaN,-20.90,-5.27,-5.87,15146,15131


In [4]:
#Piskunov (2008)
piskunov_df = pd.read_fwf("Piskunov2008.table", 
                          widths = [6,18,7,7,8,6,6,6,6,6,9,9,6,6,9,9], header = None,
                          names = ['COCD','Name','GLON[deg]','GLAT[deg]','DistMod','E(B-V)','Dist[pc]',\
                                   'logt[yr]','rt[pc]','e_rt[pc]','logM[MSun]','e_logM[MSun]','rtA[pc]','e_rtA[pc]',
                                   'logMA[MSun]','e_logMA[MSun]'])

piskunov_df.replace(-9.999,np.nan, inplace=True)
piskunov_df.replace(-9.9,np.nan, inplace=True)

#fix the IDs
piskunov_ID = piskunov_df['Name']
piskunov_ID = piskunov_ID.str.strip().str.replace(' ','_' )
piskunov_df['Name'] = piskunov_ID



piskunov_df

,COCD,Name,GLON[deg],GLAT[deg],DistMod,E(B-V),Dist[pc],logt[yr],rt[pc],e_rt[pc],logM[MSun],e_logM[MSun],rtA[pc],e_rtA[pc],logMA[MSun],e_logMA[MSun]
0,1,Berkeley_58,116.73,-1.29,14.555,0.55,3715,8.20,NaN,NaN,NaN,NaN,22.9,10.9,3.380,0.623
1,2,Berkeley_59,118.22,5.00,13.782,1.22,1000,6.80,NaN,NaN,NaN,NaN,8.0,3.5,2.221,0.564
2,3,Blanco_1,14.17,-79.02,7.180,0.01,269,8.32,22.8,3.8,3.646,0.219,20.0,2.4,3.480,0.160
3,4,Alessi_20,117.64,-3.69,8.948,0.22,450,8.22,5.4,1.6,1.742,0.391,4.0,0.8,1.362,0.250
4,5,Mayer_1,119.44,-0.93,12.015,0.40,1429,7.74,NaN,NaN,NaN,NaN,16.7,5.6,3.150,0.442
5,6,Stock_20,119.92,-0.10,10.413,0.20,909,8.53,NaN,NaN,NaN,NaN,6.6,1.3,1.974,0.260
6,7,Stock_21,120.05,-4.83,11.447,0.40,1100,8.72,NaN,NaN,NaN,NaN,8.3,2.4,2.259,0.376
7,8,NGC_129,120.27,-2.54,12.759,0.55,1625,7.87,14.9,2.7,2.984,0.240,15.2,2.7,3.011,0.235
8,9,NGC_146,120.87,0.50,13.897,0.48,3032,7.37,NaN,NaN,NaN,NaN,20.8,7.1,3.298,0.445
9,10,NGC_225,122.01,-1.08,9.925,0.27,657,8.19,NaN,NaN,NaN,NaN,5.7,1.3,1.787,0.311


In [5]:
#Kharchenko (2013)
kharchenko_df = pd.read_fwf("Kharchenko2013.table", 
                          widths = [5,18,2,1,9,8,8,8,7,7,7,7,7,7,8,8,6,6,7,7,8,7,7,7,7,7,7,7,4,8,8,8,8,8,8,5,4,8,7,4], 
                          header = None,
                          names = ['MWSC','Name','Type','n_Type','RA[hr]','Dec[deg]','GLON[deg]','GLAT[deg]',
                                   'r0[deg]','r1[deg]','r2[deg]','pmRA[mas/yr]','pmDec[mas/yr]','e_pm[mas/yr]',
                                   'RV[km/s]','e_RV[km/s]','n_RV[km/s]','N1sr0','N1sr1','N1sr2','d[pc]','E(B-V)',
                                   'appDistMod[mag]','E(J-Ks)','E(J-H)','dH','logt[yr]','e_logt[yr]','Nt','rc[pc]',
                                   'e_rc[pc]','rt[pc]','e_rt[pc]','k[pc-2]','e_k[pc-2]','Src','SType','[Fe/H][Sun]',
                                   'e_[Fe/H][Sun]','n_[Fe/H]'])

kharchenko_df['RV[km/s]'].replace(999.99,np.nan, inplace=True)
kharchenko_df['e_RV[km/s]'].replace(99.99,np.nan, inplace=True)
kharchenko_df['e_logt[yr]'].replace(0.000,np.nan, inplace=True)
kharchenko_df['Nt'].replace(-1,np.nan, inplace=True)
kharchenko_df['rc[pc]'].replace(0.00,np.nan, inplace=True)
kharchenko_df['e_rc[pc]'].replace(0.00,np.nan, inplace=True)
kharchenko_df['rt[pc]'].replace(0.00,np.nan, inplace=True)
kharchenko_df['e_rt[pc]'].replace(0.00,np.nan, inplace=True)
kharchenko_df['k[pc-2]'].replace(0.00,np.nan, inplace=True)
kharchenko_df['e_k[pc-2]'].replace(0.00,np.nan, inplace=True)
kharchenko_df['[Fe/H][Sun]'].replace(99.999,np.nan, inplace=True)
kharchenko_df['e_[Fe/H][Sun]'].replace(9.99,np.nan, inplace=True)
kharchenko_df['n_[Fe/H]'].replace(0.1,np.nan, inplace=True)

kharchenko_df.loc[kharchenko_df['Name'] == 'Skiff_J0458+43.0']

,MWSC,Name,Type,n_Type,RA[hr],Dec[deg],GLON[deg],GLAT[deg],r0[deg],r1[deg],r2[deg],pmRA[mas/yr],pmDec[mas/yr],e_pm[mas/yr],RV[km/s],e_RV[km/s],n_RV[km/s],N1sr0,N1sr1,N1sr2,d[pc],E(B-V),appDistMod[mag],E(J-Ks),E(J-H),dH,logt[yr],e_logt[yr],Nt,rc[pc],e_rc[pc],rt[pc],e_rt[pc],k[pc-2],e_k[pc-2],Src,SType,[Fe/H][Sun],e_[Fe/H][Sun],n_[Fe/H]
319,442.0,Skiff_J0458+43.0,NaN,NaN,4.9692,43.007,162.998,0.123,0.015,0.07,0.15,-2.92,-2.68,0.43,NaN,NaN,0.0,2.0,36.0,113.0,3000.0,0.4,12.514,0.192,0.128,0.0,8.7,NaN,NaN,1.12,0.31,15.03,4.66,3.81,0.92,DIAS,NaN,NaN,NaN,0.0


In [6]:
# Salaris (2004)
salaris_df = pd.read_csv('Salaris2004_viaWEBDA.txt', sep='\t', header = 15)
salaris_df

#change the Hyades to Melotte 25 so that it matches with van den Bergh for position
names = salaris_df['Name'].values
xx = np.where(names == 'Hyades')
names[xx] = 'Melotte_25'
salaris_df['Name'] = names

salaris_df

,Name,deltaV,sigdV,[FeH],sigFeH,t[Gyr],sigt[Gyr],logt,Rgc[kpc],z[pc]
0,Arp-Madore_2,2.5,0.15,0.00,0.15,7.24,1.93,0.86,14.06,-740
1,Berkeley_17,2.8,0.15,-0.33,0.15,10.06,2.77,1.00,10.89,-155
2,Berkeley_18,2.3,0.15,0.02,0.15,5.69,1.49,0.76,12.09,325
3,Berkeley_20,2.1,0.05,-0.61,0.15,4.05,0.69,0.61,16.12,-2420
4,Berkeley_21,1.6,0.25,-0.97,0.15,2.18,0.78,0.34,14.27,-255
5,Berkeley_22,2.1,0.25,-0.30,0.15,4.26,1.65,0.63,11.92,-530
6,Berkeley_29,2.1,0.05,-0.18,0.15,4.34,0.74,0.64,18.72,1465
7,Berkeley_30,0.3,0.15,0.00,0.20,0.74,0.16,-0.13,10.58,120
8,Berkeley_31,2.3,0.25,-0.40,0.15,5.32,2.11,0.73,12.02,340
9,Berkeley_32,2.4,0.15,-0.50,0.15,5.91,1.56,0.77,11.30,235


In [7]:
#van den Bergh (2006)
#there were two rows for Berkeley 69, with slightly different values.  I kept the first one.

vandenbergh_df = pd.read_csv('vandenbergh2006.tsv', sep='|', header = 49)

#fix the names
names = vandenbergh_df['SimbadName'].values
def representsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False
    
for i,x in enumerate(names):
    if (x[0:2] == 'Cl'):
        names[i] = x[2:]
    if (x[0:1] == 'N' and representsInt(x[1:1])):
        names[i] = 'NGC '+x[1:]
    names[i] = names[i].strip().replace('  ',' ').replace( ' ','_' )

vandenbergh_df['Name'] = names
vandenbergh_df

,Seq,Name,l[deg],Diam[pc],R[pc],Z[pc],E(B-V),logT[yr],SimbadName,_RA[deg],_Dec[deg]
0,1,Trumpler_31,2,1.43,986,-39,0.35,8.87,Trumpler_31,269.97500,-28.18333
1,2,NGC_6520,2,2.29,1577,-78,0.43,7.72,NGC_6520,270.85000,-27.88333
2,3,NGC_6530,6,5.42,1330,-31,0.33,6.87,NGC_6530,271.12500,-24.36667
3,4,Bochum_14,6,0.34,578,-5,1.51,7.00,Bochum_14,270.50000,-23.70000
4,5,NGC_6514,7,6.65,816,-4,0.19,7.37,NGC_6514,270.59583,-23.03000
5,6,NGC_6546,7,3.82,938,-23,0.49,7.85,NGC_6546,271.90000,-23.33333
6,7,NGC_6531,7,2.91,1205,-7,0.28,7.07,NGC_6531,271.05000,-22.48333
7,8,NGC_6494,9,5.30,628,31,0.36,8.48,NGC_6494,269.25000,-18.98333
8,9,Mrk_38,11,0.86,1471,-24,0.40,6.88,Mrk_38,169.56958,53.74861
9,10,Trumpler_33,12,2.55,1755,-99,0.36,7.68,Trumpler_33,276.17500,-19.71667


In [8]:
#Gaia DR2 from Cantat-Gaudin+, 2018
gaiaDR2_df = pd.read_csv('Cantat-Gaudin2018_GaiaDR2.tsv', sep='|', header = 56)

names = gaiaDR2_df['Name']
names = names.str.strip()

for index, row in  gaiaDR2_df.iterrows():
#for i,n in enumerate(names):
    if (names[index][0:3] == "ESO"):
        names[index] = row['SimbadName'].replace(' ','_')

gaiaDR2_df['Name'] = names

gaiaDR2_df.loc#[(gaiaDR2_df['Name'] == 'Collinder_258') | (gaiaDR2_df['Name'] == 'Harvard_5')]

In [9]:
#mwsc + webda
mwsc_webda_df = mwsc_df.join(webda_df.set_index('Name'), on='Name', 
                             how='outer', lsuffix='_mwsc', rsuffix='_webda')
      
# + piskunov
mwsc_webda_piskunov_df = mwsc_webda_df.join(piskunov_df.set_index('Name'), on='Name', 
                                            how='outer', rsuffix='_piskunov')

# + kharchenko
mwsc_webda_piskunov_kharchenko_df = mwsc_webda_piskunov_df.join(kharchenko_df.set_index('Name'), on='Name', 
                                            how='outer', rsuffix='_kharchenko')

# + Salaris
mwsc_webda_piskunov_kharchenko_salaris_df = mwsc_webda_piskunov_kharchenko_df.join(salaris_df.set_index('Name'), 
                                            on='Name', how='outer', rsuffix='_salaris')

# + vandenberg
mwsc_webda_piskunov_kharchenko_salaris_vandenbergh_df = mwsc_webda_piskunov_kharchenko_salaris_df.join(
                                            vandenbergh_df.set_index('Name'), 
                                            on='Name', how='outer', rsuffix='_vandenbergh')


# + Gaia
mwsc_webda_piskunov_kharchenko_salaris_vandenbergh_gaia_df = mwsc_webda_piskunov_kharchenko_salaris_vandenbergh_df.join(
                                            gaiaDR2_df.set_index('Name'), 
                                            on='Name', how='outer', rsuffix='_GaiaDR2')

#rename
OCs = mwsc_webda_piskunov_kharchenko_salaris_vandenbergh_gaia_df.copy()

print(len(OCs.loc[pd.isnull(OCs['Name'])]))

#reindex
OCs = OCs.reset_index(drop=True)

#there's one NaN row; drop that
idx = OCs.index[pd.isnull(OCs['Name'])]
OCs.drop(idx, inplace=True)

#reindex
OCs = OCs.reset_index(drop=True)


print(len(mwsc_df), len(webda_df), len(piskunov_df), len(kharchenko_df), len(salaris_df), len(vandenbergh_df),
     len(gaiaDR2_df), len(mwsc_webda_piskunov_kharchenko_salaris_vandenbergh_df),
      len(mwsc_webda_piskunov_kharchenko_salaris_vandenbergh_gaia_df), len(OCs))

#dump to a file
OCs.to_csv('OCcompiled.csv')

1
2908 937 650 3006 71 599 1228 3249 3336 3335


In [10]:
#a quick check to make sure that items matched up
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
print(OCs.loc[OCs['Name'] == 'NGC_2682'].iloc[0])

Name                                                                NGC_2682
broad_type                                                                  
cluster_status                                                              
ra                                                                  08 51 23
dec                                                                 +11 48.9
lii                                                                  215.691
bii                                                                   31.923
core_radius                                                              0.1
central_radius                                                          0.55
cluster_radius                                                          1.03
pm_ra                                                                  -7.31
pm_dec                                                                 -5.92
pm_tot_error                                                            0.08

## Check for missing clusters

*in case WEBDA list is not complete*

*I found 2 duplicates in David James' WEBDA file = Ruprecht 89, NGC 2414.  The NGC 2414 lines are identical.  The 2nd Ruprecht 89 line was more complete.  So I deleted the first lines of each from the file.*


In [11]:
for index, row in piskunov_df.iterrows():
    check = OCs.loc[OCs['Name'] == row['Name']]
    if (len(check['Name']) != 1):
        print('Piskunov', row['Name'])
        print('Check', check['Name'])
        print('')
        
for index, row in salaris_df.iterrows():
    check = OCs.loc[OCs['Name'] == row['Name']]
    if (len(check['Name']) != 1):
        print('Salaris', row['Name'])
        print('Check', check['Name'])
        print('')

for index, row in vandenbergh_df.iterrows():
    check = OCs.loc[OCs['Name'] == row['Name']]
    if (len(check['Name']) != 1):
        print('van den Bergh', row['Name'])
        print('Check', check['Name'])
        print('')
        
for index, row in  webda_df.iterrows():
    check = OCs.loc[OCs['Name'] == row['Name']]
    if (len(check['Name']) != 1):
        print('WEBDA', row['Name'])
        print('Check', check['Name'])
        print('')

for index, row in  mwsc_df.iterrows():
    check = OCs.loc[OCs['Name'] == row['Name']]
    if (len(check['Name']) != 1):
        print('MWSC', row['Name'])
        print('Check', check['Name'])
        print('')
        
for index, row in  gaiaDR2_df.iterrows():
    check = OCs.loc[OCs['Name'] == row['Name']]
    if (len(check['Name']) != 1):
        print('gaiaDR2_df', row['Name'])
        print('Check', check['Name'])
        print('')

In [12]:
check = OCs.loc[OCs['Name'] == 'Berkeley_69']
print(len(check['Name']))

1


In [13]:
check = OCs.loc[OCs['Name'] == 'Hyades']
print(len(check['Name']))

0


In [14]:
print(OCs.columns.values)

['Name' 'broad_type' 'cluster_status' 'ra' 'dec' 'lii' 'bii' 'core_radius'
 'central_radius' 'cluster_radius' 'pm_ra' 'pm_dec' 'pm_tot_error'
 'rad_vel' 'rad_vel_error' 'num_rad_vel_stars' 'num_core_stars'
 'num_central_stars' 'num_cluster_stars' 'distance' 'e_bv'
 'distance_modulus' 'e_jk' 'e_jh' 'delta_h' 'log_age' 'log_age_error'
 'num_log_age_stars' 'king_core_radius' 'king_core_radius_error'
 'king_tidal_radius' 'king_tidal_radius_error' 'king_norm_factor'
 'king_norm_factor_error' 'reference_code' 'cluster_type' 'metallicity'
 'metallicity_error' 'num_metallicity_stars' 'comments' 'class' 'RA_2000'
 'Dec_2000' 'l' 'b' 'Dist' 'Mod' 'EB-V' 'Age' 'ST' 'Z' 'Diam' 'Fe/H' 'MRV'
 'pm RA' 'pm Dec' 'Measures' 'Stars' 'COCD' 'GLON[deg]' 'GLAT[deg]'
 'DistMod' 'E(B-V)' 'Dist[pc]' 'logt[yr]' 'rt[pc]' 'e_rt[pc]' 'logM[MSun]'
 'e_logM[MSun]' 'rtA[pc]' 'e_rtA[pc]' 'logMA[MSun]' 'e_logMA[MSun]' 'MWSC'
 'Type' 'n_Type' 'RA[hr]' 'Dec[deg]' 'GLON[deg]_kharchenko'
 'GLAT[deg]_kharchenko' 'r0[deg]' '

## Check for duplicates in RA and Dec

In [15]:
def getCoord(row):

    if (pd.notna(row['_RAJ2000']) and pd.notna(row['_DEJ2000'])):
        return SkyCoord(ra=row['_RAJ2000']+' hours', dec=row['_DEJ2000']+' degree', frame='icrs')
    
    elif (pd.notna(row['ra']) and pd.notna(row['dec'])):
        return SkyCoord(ra=row['ra']+' hours', dec=row['dec']+' degree', frame='icrs')
        
    elif (pd.notna(row['RA_2000']) and pd.notna(row['Dec_2000'])):
        return SkyCoord(ra=row['RA_2000']+' hours', dec=row['Dec_2000']+' degree', frame='icrs')
        
    elif (pd.notna(row['RA[hr]']) and pd.notna(row['Dec[deg]'])):
        return SkyCoord(ra=row['RA[hr]']*units.hourangle, dec=row['Dec[deg]']*units.degree, frame='icrs')
        
    elif (pd.notna(row['_RA[deg]']) and pd.notna(row['_Dec[deg]'])):
        return SkyCoord(ra=row['_Dec[deg]']*units.degree, dec=row['_Dec[deg]']*units.degree, frame='icrs')
        
    elif (pd.notna(row['GLON[deg]']) and pd.notna(row['GLAT[deg]'])):
        return SkyCoord(l=row['GLON[deg]']*units.degree, b=row['GLAT[deg]']*units.degree, frame='galactic').icrs

    elif (pd.notna(row['l']) and pd.notna(row['b'])):
        return SkyCoord(l=row['l']*units.degree, b=row['b']*units.degree, frame='galactic').icrs
    
    else:
        print('NO RA, Dec', row['Name'])
        return False

In [16]:
#first get the coordinates in lists
RA = []
Dec = []
for index, row in OCs.iterrows():
    #RA and Dec
    c = getCoord(row)
    if (c):
        RA.append(c.ra.degree)
        Dec.append(c.dec.degree)

catalog = SkyCoord(ra = RA*units.degree, dec = Dec*units.degree)

In [17]:
#now match to the full catalog to see if there are duplicates
#################
# Collinder_220 is not NGC_3247 
# Luginbuhl-Skiff_1 is not Skiff_1 
#
# changed Pismis 6 to NGC 2645 in MWSC, Piskunov and Kharchenko
# Luginbuhl-Skiff_1 is Skiff_J0614+12.9 (?) changed in WEBDA
# Skiff_2 is Skiff_J0458+43.0 (?) changed in WEBDA
# PTB 9 is a planetary nebula (not an OC --though maybe in the OC NGC 7762?) -- changed label in MWSC, removed from Kharchenko
# Collinder_258 is Harvard_5 (same PM, distance, etc.), both are in Gaia table, removed Harvard_5
# FSR_1686 is Juchert_10, changed to Juchert 10 in MWSC and Kharchenko
################
max_sep = 1.0 * units.arcmin
for index, row in OCs.iterrows():
    c = getCoord(row)
    idx, d2d, d3d = c.match_to_catalog_sky(catalog, nthneighbor=2) #first neighbor is itself
    #print(index, row['Name'], OCs.iloc[int(idx)]['Name'], d2d.degree)
    if (d2d < max_sep and idx != index):
        print('!!! OVERLAP',row['Name'], OCs.iloc[int(idx)]['Name'], d2d.arcminute, index, idx)
        


# Make a plot of the age distribution and mass distribution

### First check how many have both

In [18]:

hasMass= OCs.loc[(pd.notnull(OCs['logM[MSun]'])) |
                 (pd.notnull(OCs['num_cluster_stars'])) |
                 (pd.notnull(OCs['Stars'])) | 
                 (pd.notnull(OCs['logMA[MSun]'])) |
                 (pd.notnull(OCs['N1sr0'])) |
                 (pd.notnull(OCs['N1sr1'])) |
                 (pd.notnull(OCs['N1sr2'])) 
                ]
hasAge = OCs.loc[(pd.notnull(OCs['Age'])) | 
                 (pd.notnull(OCs['log_age'])) |
                 (pd.notnull(OCs['logt[yr]'])) |
                 (pd.notnull(OCs['logt[yr]_kharchenko'])) |
                 (pd.notnull(OCs['logt'])) |
                 (pd.notnull(OCs['logt_vandenbergh'])) |
                 (pd.notnull(OCs['t[Gyr]']))
                 ]
hasBoth = OCs.loc[( 
         (pd.notnull(OCs['logM[MSun]'])) |
         (pd.notnull(OCs['num_cluster_stars'])) |
         (pd.notnull(OCs['Stars'])) | 
         (pd.notnull(OCs['logMA[MSun]'])) |
         (pd.notnull(OCs['N1sr0'])) |
         (pd.notnull(OCs['N1sr1'])) |
         (pd.notnull(OCs['N1sr2'])) 
    ) & (
        (pd.notnull(OCs['Age'])) | 
        (pd.notnull(OCs['log_age'])) |
        (pd.notnull(OCs['logt[yr]'])) |
        (pd.notnull(OCs['logt[yr]_kharchenko'])) |
        (pd.notnull(OCs['logt'])) |
        (pd.notnull(OCs['logt_vandenbergh'])) |
        (pd.notnull(OCs['t[Gyr]']))
    )]
print(len(OCs), len(hasMass), len(hasAge), len(hasBoth))

KeyError: 'logt_vandenbergh'

In [ ]:
noAge = OCs.loc[(pd.isnull(OCs['Age'])) &
                 (pd.isnull(OCs['log_age'])) &
                 (pd.isnull(OCs['logt[yr]'])) &
                 (pd.isnull(OCs['logt[yr]_kharchenko'])) &
                 (pd.isnull(OCs['logt'])) &
                 (pd.isnull(OCs['logt_vandenbergh'])) &
                 (pd.isnull(OCs['t[Gyr]']))
                 ]
noAge

In [ ]:
#Add a column to estimate the mass from the number of stars? (or vice versa) 
#This would require an estimate of the mean mass, which depends on age

#as a test, just assume <m>=0.5
meanM = 0.5

logMass = []
logAge = []
name = []
for index, row in  hasBoth.iterrows():
    name.append(row['name'])
    
    #age
#     (pd.notnull(OCs['Age'])) | 
#                  (pd.notnull(OCs['log_age'])) |
#                  (pd.notnull(OCs['logt[yr]'])) |
#                  (pd.notnull(OCs['logt[yr]_kharchenko'])) |
#                  (pd.notnull(OCs['logt'])) |
#                  (pd.notnull(OCs['logt_vandenbergh'])) |
#                  (pd.notnull(OCs['t[Gyr]']))
                 
                
    if (pd.notnull(row['log_age'])): #MWSC
        logAge.append(row['log_age'])
    elif (pd.notnull(row['logt'])): #Solaris
        logAge.append(row['logt'])
    elif (pd.notnull(row['log(t[yr])K'])): #Kharchenko
        logAge.append(row['log(t[yr])K'])
    elif (pd.notnull(row['Age'])): #WEBDA
        logAge.append(np.log10(row['Age']))

    #mass
#          (pd.notnull(OCs['logM[MSun]'])) |
#          (pd.notnull(OCs['num_cluster_stars'])) |
#          (pd.notnull(OCs['Stars'])) | 
#          (pd.notnull(OCs['logMA[MSun]'])) |
#          (pd.notnull(OCs['N1sr0'])) |
#          (pd.notnull(OCs['N1sr1'])) |
#          (pd.notnull(OCs['N1sr2'])) 
        
    if (pd.notnull(row['logM[Msun]'])): #Piskunov
        logMass.append(row['logM[Msun]'])    
    elif (pd.notnull(row['num_cluster_stars'])): #MWSC <-- NEED TO FIX THIS 
        logMass.append(np.log10(row['num_cluster_stars']*meanM))
        
print(len(name), len(logAge), len(logMass))

### Make a few plots

In [ ]:
f,(ax1, ax2) = plt.subplots(1,2)

ax1.hist(logAge, bins=40, density=True)
ax1.set_xlabel('log(Age [yr?])')
ax1.set_yscale('log')

ax2.hist(logMass, bins=40, density=True)
ax2.set_xlabel('log(Mass [Msun])')
ax2.set_yscale('log')


### As if I'm only reading from the file

In [ ]:
df = pd.read_csv("OCcompiled_hasAgeMass.csv")

data = np.vstack((df['logAge'].values, df['logMass'].values))
KDE = gaussian_kde(data)
sample = KDE.resample(size=int(1e5))

nbins = 40

f,(ax1, ax2) = plt.subplots(1,2)
ax1.hist(df['logAge'].values, bins=nbins, density=True)
ax1.hist(sample[0,:], bins=nbins, density=True, histtype='step')
ax1.set_xlabel('log(Age [yr?])')
ax1.set_yscale('log')

ax2.hist(df['logMass'].values, bins=nbins, density=True)
ax2.hist(sample[1,:], bins=nbins, density=True, histtype='step')
ax2.set_xlabel('log(Mass [Msun])')
ax2.set_yscale('log')

lt = 5
lm = 2
values = np.vstack([lt, lm])
print(KDE(values))
#NOTE: the age KDE seems to be missing the edges.  Maybe I should set those to zero automatically?

### Make a smaller file that has everything we need for the EBLSST code

Name, RA, Dec, dist[kpc], rh[pc], mass[Msun], Age[Myr], Z, sigma_v[km/s]

In [ ]:
df = pd.read_csv("OCcompiled.csv")


name = []
RA = []
Dec = []

logMass = []
logAge = []
for index, row in df.iterrows():
    name.append(row['name'])
    
    #RA
    if (pd.notnull(row['ra'])):
        RA.append(row['ra'])
    elif (pd.notnull(row['RA_2000'])):
        RA.append(row['RA_2000'])
    else:
        print('NO RA', row['name'])
        #print('\nNO RA', row)
     
    #Dec
    Dec.append(row['dec'])
    
    #age
    if (pd.notnull(row['log_age'])): #MWSC
        logAge.append(row['log_age'])
    elif (pd.notnull(row['logt'])): #Solaris
        logAge.append(row['logt'])
    elif (pd.notnull(row['log(t[yr])K'])): #Kharchenko
        logAge.append(row['log(t[yr])K'])
    elif (pd.notnull(row['Age'])): #WEBDA
        logAge.append(np.log10(row['Age']))

print(len(name),len(RA))